In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

import spacy

/kaggle/input/emotion/Emotion_final.csv


In [2]:
data = pd.read_csv('/kaggle/input/emotion/Emotion_final.csv')

In [3]:
data.head(10)

,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
5,ive been feeling a little burdened lately wasn...,sadness
6,ive been taking or milligrams or times recomme...,surprise
7,i feel as confused about life as a teenager or...,fear
8,i have been with petronas for years i feel tha...,happy
9,i feel romantic too,love


In [4]:
pd.set_option("display.max_colwidth", 240)
data.head()

,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplace i will know that it is still on the property,love
4,i am feeling grouchy,anger


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21459 entries, 0 to 21458
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Text     21459 non-null  object
 1   Emotion  21459 non-null  object
dtypes: object(2)
memory usage: 335.4+ KB


In [6]:
data.isnull().sum()

Text       0
Emotion    0
dtype: int64

In [7]:
data['Emotion'].value_counts()

Emotion
happy       7029
sadness     6265
anger       2993
fear        2652
love        1641
surprise     879
Name: count, dtype: int64

In [8]:
# Show sample
print(f"{data['Text'][0]} -> {data['Emotion'][0]}")

i didnt feel humiliated -> sadness


In [9]:
# load english language model and create nlp object from it
nlp = spacy.load("en_core_web_sm")

In [10]:
txt = data['Text'][3]
txt

'i am ever feeling nostalgic about the fireplace i will know that it is still on the property'

In [11]:
doc = nlp(txt)

In [12]:
for token in doc:
    print(token)

i
am
ever
feeling
nostalgic
about
the
fireplace
i
will
know
that
it
is
still
on
the
property


In [13]:
for token in doc:
    print(f"Word: {token} | -> {token.lemma_}")

Word: i | -> I
Word: am | -> be
Word: ever | -> ever
Word: feeling | -> feel
Word: nostalgic | -> nostalgic
Word: about | -> about
Word: the | -> the
Word: fireplace | -> fireplace
Word: i | -> I
Word: will | -> will
Word: know | -> know
Word: that | -> that
Word: it | -> it
Word: is | -> be
Word: still | -> still
Word: on | -> on
Word: the | -> the
Word: property | -> property


In [14]:
for token in doc:
    if token.is_stop or token.is_punct:
        print(token)

i
am
ever
about
the
i
will
that
it
is
still
on
the


In [15]:
# use this utility function to get the preprocessed text data
def preprocess(text):
    # remove stop words and lemmatize the text
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
    
    return " ".join(filtered_tokens)

In [16]:
print(txt)
procces_txt = preprocess(txt)
print(procces_txt)

i am ever feeling nostalgic about the fireplace i will know that it is still on the property
feel nostalgic fireplace know property


In [17]:
data['preprocessed_text'] = data['Text'].apply(preprocess)

In [18]:
data

,Text,Emotion,preprocessed_text
0,i didnt feel humiliated,sadness,nt feel humiliate
1,i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake,sadness,feel hopeless damned hopeful care awake
2,im grabbing a minute to post i feel greedy wrong,anger,m grab minute post feel greedy wrong
3,i am ever feeling nostalgic about the fireplace i will know that it is still on the property,love,feel nostalgic fireplace know property
4,i am feeling grouchy,anger,feel grouchy
...,...,...,...
21454,Melissa stared at her friend in dism,fear,Melissa stare friend dism
21455,Successive state elections have seen the governing parties pummelled by a dismayed electora,fear,successive state election see govern party pummel dismayed electora
21456,Vincent was irritated but not dismay,fear,Vincent irritated dismay
21457,Kendall-Hume turned back to face the dismayed coup,fear,Kendall Hume turn face dismayed coup


In [19]:
data['Emotion_num'] = data['Emotion'].map({'happy' : 0, 'sadness': 1, 'anger': 2, 'fear': 3, 'love':4, 'surprise':5})

data.head(10)

,Text,Emotion,preprocessed_text,Emotion_num
0,i didnt feel humiliated,sadness,nt feel humiliate,1
1,i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake,sadness,feel hopeless damned hopeful care awake,1
2,im grabbing a minute to post i feel greedy wrong,anger,m grab minute post feel greedy wrong,2
3,i am ever feeling nostalgic about the fireplace i will know that it is still on the property,love,feel nostalgic fireplace know property,4
4,i am feeling grouchy,anger,feel grouchy,2
5,ive been feeling a little burdened lately wasnt sure why that was,sadness,have feel little burden lately nt sure,1
6,ive been taking or milligrams or times recommended amount and ive fallen asleep a lot faster but i also feel like so funny,surprise,have take milligram time recommend have fall asleep lot fast feel like funny,5
7,i feel as confused about life as a teenager or as jaded as a year old man,fear,feel confused life teenager jade year old man,3
8,i have been with petronas for years i feel that petronas has performed well and made a huge profit,happy,petrona year feel petrona perform huge profit,0
9,i feel romantic too,love,feel romantic,4


In [20]:
X_train, X_test, y_train, y_test = train_test_split(data['preprocessed_text'], data['Emotion_num'], 
                                                    test_size=0.2, random_state=42, stratify=data['Emotion_num'])

In [34]:
v = TfidfVectorizer()

X_train_cv = v.fit_transform(X_train)
X_test_cv = v.transform(X_test)

 #All TfidfVectorizer vocabularies
#print(v.vocabulary_)

In [24]:
RFC_model = RandomForestClassifier()

RFC_model.fit(X_train_cv, y_train)

RandomForestClassifier()

In [25]:
# Get the predictions for X_test and store it in y_pred
y_pred = RFC_model.predict(X_test_cv)

In [26]:
# Print Accuracy
print(accuracy_score(y_test, y_pred))

0.8541472506989748


In [27]:
# Print the classfication report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.90      0.87      1406
           1       0.89      0.90      0.89      1253
           2       0.84      0.84      0.84       599
           3       0.87      0.80      0.83       530
           4       0.74      0.67      0.71       328
           5       0.82      0.71      0.76       176

    accuracy                           0.85      4292
   macro avg       0.84      0.80      0.82      4292
weighted avg       0.85      0.85      0.85      4292



In [28]:
test_text = data['Text'][2000]
test_text

'i provided dinner alcohol and a place to crash and all i got in return was the feeling of being completely unwelcome in my own apartment'

In [29]:
test_text_processed = [preprocess(test_text)]
test_text_processed

['provide dinner alcohol place crash get return feeling completely unwelcome apartment']

In [31]:
test_text_vc = v.transform(test_text_processed)

In [32]:
test_text = RFC_model.predict(test_text_vc)

In [33]:
print(f"{data['Emotion'][2000]} -> {data['Emotion_num'][2000]}")
print(test_text)

sadness -> 1
[1]
